In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: TimbralPMS – Timbral Parameter Mapping Sonification of multivariate data series

This notebook introduces and demonstrates usage of the `TimbralPMS` sonecule.
- The sonecule enables Timbral Sonification, i.e., the sonification of a multivariate data set where each channel controls the amplitude of a partial of a pitched sounds consisting of multiple harmonics.
- It can be initialized with 
  - a pya Asig (i.e. audio signal using pya)
    - `TimbralPMS(asig, sr=None, channels=None)`
  - a pandas DataFrame or Series
    - `TimbralPMS.from_df(df, sr=44100, time_column=None, columns=None)`
  - a numpy ndarray
    - `TimbralPMS.from_np(data, sr=44100, time_column=None, columns=None)`
- Preprocessing such as time stretching, slicing, filtering is offered by specialized functions, either in pya (iirfilter, stretch) or libraries such as scipy.signal - correspondingly processed signals can be passed into TimbralPMS Sonecules for audition and interaction.
- The current TimbralPMS uses the SuperCollider3 via the sc3nb backend of mesonic, and uses a single sc3 `PlayBuf` UGen operating on a $d$-channel buffer. Different from audification, here the values read from the buffer control the amplitude of a battery of (overtone) sine waves.
- The synth is mutable, i.e., its parameters can be controlled interactively (via code or GUI)
  - should enable pause/resume
  - control parameters are
    - freq: the fundamental frequency of the sound
    - rate: how fast the the Buffer is read (==1 for at sampling rate sr)
    - amp: the amplitude 
    - pan: the stereo position at which the sound appears in the mix
  - with onset (in schedule)

Let's get started. First some imports and settings and startup of sonecules

In [ ]:
# headers and imports for the demo
import sonecules as sn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx
ctx.enable_realtime();

Load data sets used for the demo

In [ ]:
%run ../data/prepare-data.ipynb

Let's use EEG data to demonstrate `TimbralPMS`

In [ ]:
df = dataframes['eeg'].loc[:, :]
df.plot(subplots=True, lw=0.5);

## Usage Demo for the TimbralPMS Sonecule

In [ ]:
from sonecules.bufferson import TimbralPMS

The following code cell shows everything needed 
- to create the sonecule with data, 
- to reset the auditory canvas (the timeline)
- to start the playback at a given rate
- to plot the timeline.

In [ ]:
# create the sonecule from data (e.g. channel 7 of the EEG data set )
tson = TimbralPMS.from_df(df, sr=256) 
# plot the data (just for fun)
plt.figure(); tson.dasig.plot(offset=1, lw=0.5)

In [ ]:
# (re)schedule the event (which is just one: to start the synth)
tson.reschedule(at=0, rate=3, freq=60, pan=0, amp=0.1, loop=0).start()

In [ ]:
# let's listen to the onset of the epilepsy in realime
ctx.reset()
tson = TimbralPMS(Asig(df.values, sr=256)[{6.2:10.4},::2]) # use even channels
tson.schedule(at=0, rate=1, freq=50, startpos=0, trfreq=0, pan=0, amp=0.1, loop=1).start()
plt.figure(); tson.dasig.plot(offset=1, lw=0.5)

In [ ]:
# we can set startPos (Attention: in samples, not duration!) and trfreq to make a selection
tson.set(trfreq=0.1, rate=0.5, startpos=256*1.8, amp=0.3)

In [ ]:
tson.set(freq=40, rate=0.25)

In [ ]:
# or interact with a GUI
from ipywidgets import interactive
def xplore(startpos=256):
    tson.set(rate=0, startpos=startpos, trfreq=20)
interactive(xplore, startpos=(0, 1000, 1))

In [ ]:
tson.stop()

Next demonstration shows with how few lines of code you can probe the timbre:

Simply execute the cell and move the mouse pointer along the x-axis in the plot

In [ ]:
# or probe the plot interactively 
tson = TimbralPMS(Asig(df.values, sr=256)[{6.2:10.4},::]) # use even channels
ctx.timeline.reset()
tson.schedule(at=0, rate=0, freq=50, startpos=0, trfreq=30, pan=0, amp=0.1, loop=1).start()

# and now the GUI
fig = plt.figure()
ax = tson.dasig.plot(offset=1)
def on_motion(event):
    try: tson.set(startpos=event.xdata * tson.dasig.sr)
    except: pass
cid = fig.canvas.mpl_connect('motion_notify_event', on_motion)

In [ ]:
# stop the sonification
tson.stop()

In [ ]:
# Remember that you can always use stop all playing synth and the playback of the backend ctx by
ctx.stop()

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.
* It is assumed that your data is stored in an `Asig` dasig

In [ ]:
# create or load your data
a1 = Asig(dataframes['ecg'].values, sr=200)

# alternatively (see pandas documenation)
# data = pd.read_csv("your_csv_file.csv", delimiter=",")
# data = pd.read_excel("your_excel_file.xlsc") 
# and a1 = Asig(data)
plt.figure();a1.plot(offset=1)

In [ ]:
# reset the timeline
# only neccessary here if you are going to run this cell multiple times
# because on every execution a TimbralPMS is created and scheduled and this clutters the timeline
ctx.timeline.reset() 

# load your data / select your data
myasig = Asig(dataframes['ecg'].values, sr=200)

# any preprocessing here: e.g. if we want warped data 
myasig.sig = np.abs(a1.sig)**0.5

# sonecule for your synth with defaults and bounds
tson = TimbralPMS(myasig)

# finally start the realtime playback at a given rate
tson.schedule(at=0, rate=0.5, freq=100).start();

In [ ]:
# execute stop in case you don't want the sonification to continue
tson.stop()

In [ ]:
ctx.close()  # close the mesonic context, exits backend gracefully